# Study ML Results

## Load Data

Load libraries:

In [193]:
import json
import pathlib
import pandas as pd
from typing import Dict, Any

Define constants:

In [194]:
RESULTS_PATH = "../../results/"
DATE_EXPERIMENTS = "20240604_1409"
LINEAR_REGRESSION_FILENAMME = "linear_regression.json"
DECISION_TREE_FILENAME = "decision_tree_regressor.json"
KNN_FILENAME = "knn_regressor.json"
XGBOOST_SINGLE_OUTPUT_FILENAME = "xgboost_single_output_regressor.json"
XGBOOST_MULTI_OUTPUT_FILENAME = "xgboost_multi_output_regressor.json"

Load data:

In [195]:
experiments_path = pathlib.Path(RESULTS_PATH) / DATE_EXPERIMENTS

def load_experiment_results(file_path: pathlib.Path) -> Dict[str, Any]:
    """
    Load JSON data from a file.

    Args:
        file_path (pathlib.Path): Path to the JSON file.

    Returns:
        Dict[str, Any]: The loaded JSON data.
    """
    with file_path.open('r') as file:
        return json.load(file)


linear_regression_results = load_experiment_results(
    experiments_path / LINEAR_REGRESSION_FILENAMME
)
decision_tree_results = load_experiment_results(
    experiments_path / DECISION_TREE_FILENAME
)
knn_results = load_experiment_results(experiments_path / KNN_FILENAME)
xgboost_single_output_results = load_experiment_results(
    experiments_path / XGBOOST_SINGLE_OUTPUT_FILENAME
)
xgboost_multi_output_results = load_experiment_results(
    experiments_path / XGBOOST_MULTI_OUTPUT_FILENAME
)

Convert to dataframe:

In [196]:
def create_metrics_dataframe(results: Dict[str, Any]) -> pd.DataFrame:
    """
    Create a DataFrame with single-level columns from experiment results.

    Args:
        results (Dict[str, Any]): Dictionary containing the experiment results.

    Returns:
        pd.DataFrame: A DataFrame with single-level columns (formatted as {colname}_{metric}).
    """
    data = results["dataset_metrics"]

    rows = []
    index = []
    columns = set()

    for dataset_name, dataset_metrics in data.items():
        index.append(dataset_name)
        row = {}
        for col_name, metrics in dataset_metrics["metrics"].items():
            # Extract uppercase letters from column name
            col_name_processed = "".join(filter(str.isupper, col_name))
            for metric_name, metric_value in metrics.items():
                col_metric_name = f"{col_name_processed}_{metric_name}"
                row[col_metric_name] = metric_value
                columns.add(col_metric_name)
        rows.append(row)

    # Convert to DataFrame
    df = pd.DataFrame(rows, index=index)

    # Add boolean columns dummy, shifts and adj
    # Columns are True if the dataset name contains the respective string
    df["dummy"] = df.index.str.contains("dummy")
    df["shifts"] = df.index.str.contains("shifts")
    df["adj"] = df.index.str.contains("adj")

    # Add categorical column type
    # if the strings constains basic then 0
    # if the strings not constains no_ist then 1
    # if the strings constains no_ist then 2
    # if the strings constains complete then 3
    df["type"] = None
    df.loc[df.index.str.contains("basic"), "type"] = 0
    df.loc[
        (df.index.str.contains("ist")) & (~df.index.str.contains("no_ist")), "type"
    ] = 1
    df.loc[df.index.str.contains("no_ist"), "type"] = 2
    df.loc[df.index.str.contains("complete"), "type"] = 3

    # Ensure all columns are present and sort them alphabetically
    df = df.reindex(columns=sorted(df.columns), fill_value=None)
    return df

In [197]:
linear_regression_df = create_metrics_dataframe(linear_regression_results)
decision_tree_df = create_metrics_dataframe(decision_tree_results)
knn_df = create_metrics_dataframe(knn_results)
xgboost_single_output_df = create_metrics_dataframe(xgboost_single_output_results)
xgboost_multi_output_df = create_metrics_dataframe(xgboost_multi_output_results)

Add auxiliary function to highlight the maximum value in a column:

In [198]:
def highlight_max_min(data: pd.DataFrame) -> pd.DataFrame:
    """
    Highlight the maximum value in each column for R^2 and the minimum value in each column for RMSE.

    Args:
        data (pd.DataFrame): DataFrame to highlight.

    Returns:
        pd.DataFrame: DataFrame with highlighted values.
    """
    def highlight_values(column):
        is_r2 = "R^2" in column.name
        is_rmse = "RMSE" in column.name
        if not is_r2 and not is_rmse:
            return ['' for _ in column]
        if is_r2:
            highlight_value = column.max()
        if is_rmse:
            highlight_value = column.min()
            
        return ['background: yellow' if v == highlight_value else 'background: #185ed7' if v >= 0.2 and v != highlight_value and is_r2 else '' for v in column]
    
    return data.style.apply(highlight_values, axis=0)

In [199]:
def count_max_min_highlights_and_podiums(data: pd.DataFrame) -> pd.DataFrame:
    """
    Count the number of maximum R^2 and minimum RMSE for each dataset and sort the results.
    Additionally, count the top 3 values for R^2 and the lowest 3 values for RMSE.

    Args:
        data (pd.DataFrame): DataFrame containing the metrics.

    Returns:
        pd.DataFrame: DataFrame with counts of max R^2, min RMSE, and podium finishes per dataset, sorted by the total count.
    """
    count_df = pd.DataFrame(index=data.index, columns=['max_R2_count', 'min_RMSE_count', 'R2_podium_count', 'RMSE_podium_count'])
    count_df['max_R2_count'] = 0
    count_df['min_RMSE_count'] = 0
    count_df['R2_podium_count'] = 0
    count_df['RMSE_podium_count'] = 0
    count_df['R2_valid'] = 0

    for column in data.columns:
        if "R^2" in column:
            max_value = data[column].max()
            count_df['max_R2_count'] += data[column] == max_value
            top_3_r2 = data[column].nlargest(3).values
            count_df['R2_podium_count'] += data[column].isin(top_3_r2)
            count_df['R2_valid'] += data[column] > 0.2
        elif "RMSE" in column:
            min_value = data[column].min()
            count_df['min_RMSE_count'] += data[column] == min_value
            top_3_rmse = data[column].nsmallest(3).values
            count_df['RMSE_podium_count'] += data[column].isin(top_3_rmse)

    count_df['total_count'] = count_df['max_R2_count'] + count_df['min_RMSE_count']
    count_df['total_podiums'] = count_df['R2_podium_count'] + count_df['RMSE_podium_count']
    sorted_count_df = count_df.sort_values(by=['total_count', 'total_podiums'], ascending=False)
    return sorted_count_df

## Linear Regression

See the results:

In [200]:
linear_regression_highlighted_df = highlight_max_min(linear_regression_df)
linear_regression_highlighted_df

,CDC_RMSE,CDC_R^2,CS_RMSE,CS_R^2,CUP_RMSE,CUP_R^2,ECPGUANYEM_RMSE,ECPGUANYEM_R^2,ERC_RMSE,ERC_R^2,OTH_RMSE,OTH_R^2,PP_RMSE,PP_R^2,PSC_RMSE,PSC_R^2,VOX_RMSE,VOX_R^2,adj,dummy,shifts,type
basic,13.108765,0.000000,13.812827,-57.485107,4.958795,-1.254258,6.306477,-1.711654,8.550081,-0.645683,1.422144,-0.924748,4.603915,-1.703169,5.488559,0.236516,1.213344,0.705913,False,False,False,0
basic_adj,2.035616,0.000000,1.297133,0.484215,1.804560,0.701502,1.459422,0.854782,1.861985,0.921948,0.441670,0.814349,1.030653,0.864556,1.363938,0.952841,0.974992,0.810137,True,False,False,0
basic_dummy,182736950622.516296,0.000000,73588623502.017105,-403095924557341261824.000000,12257766942.734631,-51792413653368979456.000000,2417367951.512489,-301935116222170816.000000,46685551196.790062,-28317771561544572928.000000,50760978738.980721,-1751492697754193362944.000000,104601051337.838333,-2958896840166711754752.000000,470782865043.051758,-4703366681425419960320.000000,9446285977.997900,-44174243795176169472.000000,False,True,False,0
basic_dummy_adj,87298158024.049103,0.000000,39574803442.540741,-116575929848498388992.000000,10636482396.268721,-38990424617992642560.000000,112732235997.892044,-656705488129988624384.000000,194092315495.111084,-489432905965308805120.000000,26749279590.767857,-486473131689041199104.000000,223940549212.044739,-13560448984217628442624.000000,202204112189.937256,-867821667418416087040.000000,204146626.142018,-20632083831855088.000000,True,True,False,0
basic_shifts_3,7.961561,0.000000,14.963321,-67.633497,6.154034,-2.471930,15.824807,-16.074089,8.135234,-0.489862,1.058868,-0.067015,3.345187,-0.427116,8.999292,-1.052579,2.104707,0.115107,False,False,True,0
basic_shifts_3_adj,2.760815,0.000000,2.267505,-0.576144,1.530232,0.785359,3.407564,0.208325,2.339258,0.876806,0.474149,0.786041,1.395379,0.751732,2.292908,0.866726,0.980738,0.807892,True,False,True,0
basic_shifts_3_dummy,1336944254206.253662,0.000000,246504682830.031586,-802107779640154783744.000000,284967203579.642578,-14908917848873765961728.000000,433520344645.518372,-24565830827850017538048.000000,65064275067.514885,-109032510365452943360.000000,35190464598.411797,-170260631989355249664.000000,1402380368074.560303,-605534782198463172968448.000000,1041743676446.226929,-42843343362991056748544.000000,285326441.180413,0.000000,False,True,True,0
basic_shifts_3_dummy_adj,5.129867,0.000000,4.771934,0.699434,1.540647,0.564298,3.470314,-0.574495,6.474384,-0.079592,1.723516,0.591646,2.052015,-0.296283,2.763613,0.698453,0.083506,0.000000,True,True,True,0
complete,13.661693,0.000000,13.727114,-56.761525,4.904627,-1.205277,6.270167,-1.680519,8.747550,-0.722577,1.411996,-0.897378,4.669197,-1.780372,5.463109,0.243580,1.210587,0.707248,False,False,False,3
complete_adj,2.108698,0.000000,1.304098,0.478661,1.809431,0.699888,1.463062,0.854057,1.862828,0.921877,0.440045,0.815713,1.029153,0.864949,1.363043,0.952903,0.971739,0.811401,True,False,False,3


See the best datasets:

In [201]:
linear_regression_counts = count_max_min_highlights_and_podiums(linear_regression_df)
linear_regression_counts

,max_R2_count,min_RMSE_count,R2_podium_count,RMSE_podium_count,R2_valid,total_count,total_podiums
ist_adj,4,4,6,5,8,8,11
basic_adj,2,2,4,5,8,4,9
complete_dummy_adj,3,0,3,1,8,3,4
complete_adj,2,0,6,7,8,2,13
no_ist_dummy_adj,1,1,3,1,8,2,4
basic_shifts_3_adj,2,0,2,1,7,2,3
complete_shifts_3_adj,1,1,2,1,7,2,3
complete_shifts_3_dummy_adj,1,1,1,1,4,2,2
no_ist_adj,1,0,3,3,8,1,6
basic_shifts_3_dummy_adj,1,0,2,0,4,1,2


Let's get the names of those datasets that have less than 4 valid $R^2$ values:

In [202]:
linear_regression_invalid_datasets = linear_regression_counts[
    linear_regression_counts["R2_valid"] < 4
].index.to_list()
linear_regression_invalid_datasets

['basic',
 'basic_dummy',
 'basic_dummy_adj',
 'basic_shifts_3',
 'basic_shifts_3_dummy',
 'complete',
 'complete_dummy',
 'complete_shifts_3',
 'complete_shifts_3_dummy',
 'ist',
 'ist_dummy',
 'ist_dummy_adj',
 'ist_shifts_3',
 'ist_shifts_3_dummy',
 'no_ist',
 'no_ist_dummy',
 'no_ist_shifts_3',
 'no_ist_shifts_3_dummy']

In [244]:
linear_regression_best_datasets = linear_regression_counts[
    linear_regression_counts["R2_valid"] > 4
].index.to_list()
linear_regression_best_datasets

['basic_adj',
 'basic_shifts_3_adj',
 'complete_adj',
 'complete_dummy_adj',
 'complete_shifts_3_adj',
 'ist_adj',
 'ist_shifts_3_adj',
 'no_ist_adj',
 'no_ist_dummy_adj',
 'no_ist_shifts_3_adj']

In [203]:
linear_regression_counts = linear_regression_df.merge(
    linear_regression_counts, left_index=True, right_index=True
)

Analyze by dataset's completness:

In [204]:
linear_regression_counts[["type", "total_count", "total_podiums", "R2_valid"]].groupby(
    "type"
).agg({"total_count": "sum", "total_podiums": "sum", "R2_valid": "mean"})

,total_count,total_podiums,R2_valid
type,,,
0,12,19,2.625
1,15,20,2.625
2,9,18,3.750
3,13,26,3.875


Analyze by presence of dummy variables:

In [205]:
linear_regression_counts[["dummy", "total_count", "total_podiums", "R2_valid"]].groupby(
    "dummy"
).agg({"total_count": "sum", "total_podiums": "sum", "R2_valid": "mean"})

,total_count,total_podiums,R2_valid
dummy,,,
False,29,57,4.1875
True,20,26,2.2500


Analyze by number of shifted variables:

In [206]:
linear_regression_counts[
    ["shifts", "total_count", "total_podiums", "R2_valid"]
].groupby("shifts").agg(
    {"total_count": "sum", "total_podiums": "sum", "R2_valid": "mean"}
)

,total_count,total_podiums,R2_valid
shifts,,,
False,30,57,3.7500
True,19,26,2.6875


Analyze by presence of adjacenct variables:

In [207]:
linear_regression_counts[["adj", "total_count", "total_podiums", "R2_valid"]].groupby(
    "adj"
).agg({"total_count": "sum", "total_podiums": "sum", "R2_valid": "mean"})

,total_count,total_podiums,R2_valid
adj,,,
False,16,16,0.7500
True,33,67,5.6875


## K-Nearest Neighbors Regression

See the results:

In [208]:
knn_highlighted_df = highlight_max_min(knn_df)
knn_highlighted_df

,CDC_RMSE,CDC_R^2,CS_RMSE,CS_R^2,CUP_RMSE,CUP_R^2,ECPGUANYEM_RMSE,ECPGUANYEM_R^2,ERC_RMSE,ERC_R^2,OTH_RMSE,OTH_R^2,PP_RMSE,PP_R^2,PSC_RMSE,PSC_R^2,VOX_RMSE,VOX_R^2,adj,dummy,shifts,type
basic,17.273929,0.000000,5.270261,-7.514210,5.645406,-1.921740,3.950214,-0.063903,2.728067,0.832461,1.708600,-1.778227,2.923085,-0.089686,4.265914,0.538781,3.391580,-1.297798,False,False,False,0
basic_adj,8.069779,0.000000,4.876885,-6.290961,5.298776,-1.573654,3.680839,0.076254,3.005672,0.796616,1.435210,-0.960342,2.650780,0.104050,4.793271,0.417578,2.982721,-0.776905,True,False,False,0
basic_dummy,20.895409,0.000000,5.219388,-1.027805,2.675369,-1.467238,5.361412,-0.485205,8.951638,-0.041116,1.668545,-0.892448,2.111984,-0.206254,7.709621,-0.261344,2.972565,-3.374310,False,True,False,0
basic_dummy_adj,7.553551,0.000000,3.880084,-0.120608,2.087247,-0.501446,5.468840,-0.545485,6.254861,0.491710,3.949191,-9.603540,2.641881,-0.887275,8.151653,-0.410399,2.985107,-3.411421,True,True,False,0
basic_shifts_3,19.622448,0.000000,14.094688,-59.896323,5.674494,-1.951926,6.554042,-1.928729,3.014144,0.795481,1.729849,-1.847757,3.241059,-0.339654,3.126815,0.752208,4.418884,-2.900614,False,False,True,0
basic_shifts_3_adj,13.700863,0.000000,8.868485,-23.110075,5.816569,-2.101221,6.317843,-1.721427,3.430169,0.735111,1.693118,-1.728196,3.237711,-0.336635,2.647027,0.822381,4.300245,-2.693390,True,False,True,0
basic_shifts_3_dummy,23.140274,0.000000,14.979679,-1.962016,3.053741,-0.712067,8.591873,-8.649131,12.205487,-2.836905,4.853735,-2.239041,4.807005,-6.114701,3.152311,0.607699,0.083705,0.000000,False,True,True,0
basic_shifts_3_dummy_adj,10.575473,0.000000,15.845156,-2.313938,4.478000,-2.680881,7.716684,-6.785117,7.830433,-0.579190,5.137705,-2.628645,4.552259,-5.379586,3.391989,0.545735,0.111753,0.000000,True,True,True,0
complete,13.456629,0.000000,6.828834,-13.294642,4.007980,-0.472660,2.571578,0.549121,6.473118,0.056737,0.945909,0.148499,2.004282,0.487686,3.820354,0.630095,4.125613,-2.400045,False,False,False,3
complete_adj,8.234790,0.000000,6.175269,-10.689917,4.460253,-0.823551,2.426557,0.598542,4.816595,0.477707,0.720247,0.506299,1.994035,0.493007,3.780908,0.637618,3.921997,-2.072226,True,False,False,3


In [209]:
knn_counts = count_max_min_highlights_and_podiums(knn_df)
knn_counts

,max_R2_count,min_RMSE_count,R2_podium_count,RMSE_podium_count,R2_valid,total_count,total_podiums
complete_adj,4,3,4,3,5,7,7
complete_dummy,2,1,3,2,1,3,5
complete_dummy_adj,2,1,3,2,1,3,5
basic,2,1,2,1,2,3,3
complete_shifts_3_dummy,2,1,2,1,1,3,3
ist_shifts_3_adj,2,1,2,1,2,3,3
complete_shifts_3_dummy_adj,2,0,2,1,1,2,3
no_ist_shifts_3_dummy,2,0,2,1,1,2,3
basic_dummy_adj,1,1,1,1,1,2,2
basic_shifts_3_dummy,2,0,2,0,1,2,2


Let's get the names of those datasets that have less than 4 valid $R^2$ values:

In [210]:
knn_invalid_datasets = knn_counts[
    knn_counts["R2_valid"] < 4
].index.to_list()
knn_invalid_datasets

['complete_dummy',
 'complete_dummy_adj',
 'basic',
 'complete_shifts_3_dummy',
 'ist_shifts_3_adj',
 'complete_shifts_3_dummy_adj',
 'no_ist_shifts_3_dummy',
 'basic_dummy_adj',
 'basic_shifts_3_dummy',
 'basic_shifts_3_dummy_adj',
 'ist_shifts_3_dummy',
 'ist_shifts_3_dummy_adj',
 'no_ist_shifts_3_dummy_adj',
 'complete',
 'ist_shifts_3',
 'basic_shifts_3_adj',
 'ist',
 'no_ist_dummy',
 'no_ist_dummy_adj',
 'basic_adj',
 'ist_dummy_adj',
 'basic_dummy',
 'basic_shifts_3',
 'complete_shifts_3',
 'complete_shifts_3_adj',
 'ist_adj',
 'ist_dummy',
 'no_ist',
 'no_ist_shifts_3',
 'no_ist_shifts_3_adj']

In [245]:
knn_best_datasets = knn_counts[
    knn_counts["R2_valid"] > 4
].index.to_list()
knn_best_datasets

['complete_adj', 'no_ist_adj']

In [211]:
knn_counts = knn_df.merge(
    knn_counts, left_index=True, right_index=True
)

Analyze by dataset's completness:

In [212]:
knn_counts[["type", "total_count", "total_podiums", "R2_valid"]].groupby("type").agg(
    {"total_count": "sum", "total_podiums": "sum", "R2_valid": "mean"}
)

,total_count,total_podiums,R2_valid
type,,,
0,13,16,1.375
1,12,19,1.625
2,10,21,2.250
3,21,32,2.250


Analyze by presence of dummy variables:

In [213]:
knn_counts[["dummy", "total_count", "total_podiums", "R2_valid"]].groupby("dummy").agg(
    {"total_count": "sum", "total_podiums": "sum", "R2_valid": "mean"}
)

,total_count,total_podiums,R2_valid
dummy,,,
False,26,47,2.875
True,30,41,0.875


Analyze by number of shifted variables:

In [214]:
knn_counts[["shifts", "total_count", "total_podiums", "R2_valid"]].groupby(
    "shifts"
).agg({"total_count": "sum", "total_podiums": "sum", "R2_valid": "mean"})

,total_count,total_podiums,R2_valid
shifts,,,
False,29,53,2.00
True,27,35,1.75


Analyze by presence of adjacenct variables:

In [215]:
knn_counts[["adj", "total_count", "total_podiums", "R2_valid"]].groupby("adj").agg(
    {"total_count": "sum", "total_podiums": "sum", "R2_valid": "mean"}
)

,total_count,total_podiums,R2_valid
adj,,,
False,25,42,1.6875
True,31,46,2.0625


## Decision Tree Regression

See the results:

In [216]:
decision_tree_highlighted_df = highlight_max_min(decision_tree_df)
decision_tree_highlighted_df

,CDC_RMSE,CDC_R^2,CS_RMSE,CS_R^2,CUP_RMSE,CUP_R^2,ECPGUANYEM_RMSE,ECPGUANYEM_R^2,ERC_RMSE,ERC_R^2,OTH_RMSE,OTH_R^2,PP_RMSE,PP_R^2,PSC_RMSE,PSC_R^2,VOX_RMSE,VOX_R^2,adj,dummy,shifts,type
basic,5.983002,0.000000,5.201733,-7.294233,4.682818,-1.010323,6.183077,-1.606574,5.829618,0.234957,1.108808,-0.170036,2.447745,0.235899,7.589988,-0.460042,3.508544,-1.459017,False,False,False,0
basic_adj,8.358506,0.000000,5.297528,-7.602925,5.089108,-1.374009,4.945702,-0.667688,6.354182,0.091023,1.385036,-0.825673,3.335527,-0.418618,7.878373,-0.573428,2.758687,-0.520000,True,False,False,0
basic_dummy,18.570479,0.000000,3.823646,-0.088284,1.907491,-0.254205,6.681338,-1.306508,12.210520,-0.937146,1.645350,-0.840200,4.185753,-3.738107,6.875590,-0.003200,2.772237,-2.804588,False,True,False,0
basic_dummy_adj,36.292524,0.000000,4.822965,-0.731409,5.101918,-7.970752,6.992871,-1.526885,20.543102,-4.482878,1.643818,-0.837139,2.854881,-1.203864,11.181889,-1.653877,2.997683,-3.448670,True,True,False,0
basic_shifts_3,7.935193,0.000000,3.619578,-3.016021,4.403750,-0.777856,4.134108,-0.165264,8.820768,-0.751535,1.394397,-0.850375,3.458709,-0.525621,4.762696,0.425105,2.969567,-0.761546,False,False,True,0
basic_shifts_3_adj,5.342414,0.000000,2.902040,-1.581703,4.264176,-0.666745,3.594235,0.119211,9.245836,-0.924536,1.088694,-0.128009,3.470699,-0.535926,5.309253,0.285437,2.982380,-0.776498,True,False,True,0
basic_shifts_3_dummy,14.532169,0.000000,14.230866,-1.673284,3.247638,-0.936384,8.498529,-8.440609,12.766745,-3.197892,4.559064,-1.857693,6.265241,-11.086013,4.854382,0.069686,0.002513,0.000000,False,True,True,0
basic_shifts_3_dummy_adj,20.543716,0.000000,13.084694,-1.259844,2.409279,-0.065510,5.834363,-3.450314,12.905119,-3.289304,4.767317,-2.124311,4.364949,-4.865392,5.141420,-0.043679,0.002481,0.000000,True,True,True,0
complete,15.087023,0.000000,5.895352,-9.653679,5.096292,-1.381003,4.647130,-0.472415,4.849657,0.470546,1.438922,-0.970431,2.983448,-0.135156,6.835912,-0.184339,2.449228,-0.198302,False,False,False,3
complete_adj,7.452532,0.000000,3.147023,-2.035984,4.047673,-0.501792,3.954034,-0.065957,6.750506,-0.025903,2.385216,-4.414477,3.461888,-0.528138,4.907409,0.389510,3.934515,-2.091869,True,False,False,3


In [217]:
decision_tree_counts = count_max_min_highlights_and_podiums(decision_tree_df)
decision_tree_counts

,max_R2_count,min_RMSE_count,R2_podium_count,RMSE_podium_count,R2_valid,total_count,total_podiums
basic_shifts_3_adj,3,2,3,3,1,5,6
no_ist_shifts_3_dummy,3,1,4,2,1,4,6
basic_dummy,2,1,2,1,0,3,3
complete,2,1,2,1,1,3,3
no_ist_shifts_3,2,1,2,1,1,3,3
basic_shifts_3_dummy_adj,2,0,3,1,0,2,4
complete_shifts_3,2,0,2,2,1,2,4
no_ist_shifts_3_adj,1,1,2,2,0,2,4
basic_shifts_3_dummy,2,0,2,1,0,2,3
complete_shifts_3_dummy_adj,2,0,2,1,1,2,3


Let's get the names of those datasets that have less than 4 valid $R^2$ values:

In [218]:
decision_tree_invalid_datasets = decision_tree_counts[
    decision_tree_counts["R2_valid"] < 4
].index.to_list()
decision_tree_invalid_datasets

['basic_shifts_3_adj',
 'no_ist_shifts_3_dummy',
 'basic_dummy',
 'complete',
 'no_ist_shifts_3',
 'basic_shifts_3_dummy_adj',
 'complete_shifts_3',
 'no_ist_shifts_3_adj',
 'basic_shifts_3_dummy',
 'complete_shifts_3_dummy_adj',
 'ist_dummy_adj',
 'ist_shifts_3_dummy_adj',
 'no_ist_shifts_3_dummy_adj',
 'complete_shifts_3_dummy',
 'ist_shifts_3_dummy',
 'no_ist_adj',
 'basic',
 'ist_adj',
 'complete_adj',
 'no_ist',
 'basic_shifts_3',
 'ist_dummy',
 'ist_shifts_3',
 'ist_shifts_3_adj',
 'no_ist_dummy',
 'no_ist_dummy_adj',
 'basic_adj',
 'basic_dummy_adj',
 'complete_dummy',
 'complete_dummy_adj',
 'complete_shifts_3_adj',
 'ist']

In [246]:
decision_tree_best_datasets = decision_tree_counts[
    decision_tree_counts["R2_valid"] > 4
].index.to_list()
decision_tree_best_datasets

[]

In [219]:
decision_tree_counts = decision_tree_df.merge(
    decision_tree_counts, left_index=True, right_index=True
)

Analyze by dataset's completness:

In [220]:
decision_tree_counts[["type", "total_count", "total_podiums", "R2_valid"]].groupby(
    "type"
).agg({"total_count": "sum", "total_podiums": "sum", "R2_valid": "mean"})

,total_count,total_podiums,R2_valid
type,,,
0,16,26,0.500
1,11,19,0.375
2,16,25,0.500
3,13,18,0.500


Analyze by presence of dummy variables:

In [221]:
decision_tree_counts[["dummy", "total_count", "total_podiums", "R2_valid"]].groupby(
    "dummy"
).agg({"total_count": "sum", "total_podiums": "sum", "R2_valid": "mean"})

,total_count,total_podiums,R2_valid
dummy,,,
False,27,47,0.6875
True,29,41,0.2500


Analyze by number of shifted variables:

In [222]:
decision_tree_counts[["shifts", "total_count", "total_podiums", "R2_valid"]].groupby(
    "shifts"
).agg({"total_count": "sum", "total_podiums": "sum", "R2_valid": "mean"})

,total_count,total_podiums,R2_valid
shifts,,,
False,22,38,0.3750
True,34,50,0.5625


Analyze by presence of adjacenct variables:

In [223]:
decision_tree_counts[["adj", "total_count", "total_podiums", "R2_valid"]].groupby(
    "adj"
).agg({"total_count": "sum", "total_podiums": "sum", "R2_valid": "mean"})

,total_count,total_podiums,R2_valid
adj,,,
False,29,45,0.6250
True,27,43,0.3125


## XGBoost Single-Output Regression

See the results:

In [224]:
xgboost_single_output_highlighted_df = highlight_max_min(xgboost_single_output_df)
xgboost_single_output_highlighted_df

,CDC_RMSE,CDC_R^2,CS_RMSE,CS_R^2,CUP_RMSE,CUP_R^2,ECPGUANYEM_RMSE,ECPGUANYEM_R^2,ERC_RMSE,ERC_R^2,OTH_RMSE,OTH_R^2,PP_RMSE,PP_R^2,PSC_RMSE,PSC_R^2,VOX_RMSE,VOX_R^2,adj,dummy,shifts,type
basic,5.778510,0.000000,5.605069,-8.630351,4.550360,-0.898204,3.872283,-0.022339,3.273822,0.758723,0.933075,0.171448,1.821400,0.576913,3.658543,0.660766,1.506321,0.546745,False,False,False,0
basic_adj,0.878470,0.000000,1.962937,-0.181170,1.837219,0.690600,1.726324,0.796809,2.545937,0.854075,0.618944,0.635411,1.405696,0.748047,2.272920,0.869039,1.425858,0.593939,True,False,False,0
basic_dummy,6.973349,0.000000,4.045393,-0.218172,4.762538,-6.818445,6.064964,-0.900573,15.472059,-2.110215,5.121569,-16.830123,3.597576,-2.500078,12.509509,-2.320841,4.288959,-8.106483,False,True,False,0
basic_dummy_adj,1.442228,0.000000,1.937727,0.720516,1.390817,0.333344,2.808871,0.592303,4.996068,0.675710,1.059833,0.236323,1.219797,0.597669,2.766423,0.837562,1.265578,0.207067,True,True,False,0
basic_shifts_3,8.771249,0.000000,3.069299,-1.887744,4.403223,-0.777430,3.885481,-0.029320,5.044191,0.427219,1.984397,-2.747521,1.698914,0.631904,3.323659,0.720027,1.572540,0.506018,False,False,True,0
basic_shifts_3_adj,1.314947,0.000000,1.973572,-0.194004,1.854025,0.684913,2.028744,0.719383,3.349825,0.747374,0.680231,0.559634,1.563446,0.688325,1.758650,0.921597,1.709627,0.416230,True,False,True,0
basic_shifts_3_dummy,8.387128,0.000000,15.976511,-2.369351,2.757153,-0.395655,3.127450,-0.278478,12.206569,-2.837585,2.736660,-0.029689,2.893776,-1.578322,6.233435,-0.533969,4.960670,0.000000,False,True,True,0
basic_shifts_3_dummy_adj,7.755222,0.000000,10.337865,-0.410630,1.919563,0.323624,2.270796,0.325845,7.358902,-0.394726,3.728862,-0.911431,1.581824,0.229709,2.596807,0.733756,0.554728,0.000000,True,True,True,0
complete,5.769476,0.000000,5.073814,-6.891311,4.304056,-0.698271,3.739526,0.046559,3.933774,0.651643,1.160628,-0.281955,1.855339,0.561000,3.860012,0.622375,1.588096,0.496197,False,False,False,3
complete_adj,0.861656,0.000000,1.952748,-0.168939,1.797499,0.703833,1.667435,0.810436,2.494640,0.859896,0.618397,0.636055,1.404640,0.748426,2.249663,0.871705,1.384361,0.617230,True,False,False,3


In [225]:
xgboost_single_output_counts = count_max_min_highlights_and_podiums(xgboost_single_output_df)
xgboost_single_output_counts

,max_R2_count,min_RMSE_count,R2_podium_count,RMSE_podium_count,R2_valid,total_count,total_podiums
complete_adj,4,3,7,4,7,7,11
ist_shifts_3_adj,2,2,2,2,7,4,4
no_ist_adj,3,0,7,4,7,3,11
ist_adj,2,1,3,2,7,3,5
ist_dummy_adj,2,1,2,3,7,3,5
basic_dummy_adj,1,1,1,1,8,2,2
complete_shifts_3_dummy_adj,1,1,1,1,4,2,2
basic_adj,1,0,5,2,7,1,7
no_ist_dummy_adj,1,0,2,2,7,1,4
basic_shifts_3_adj,1,0,2,1,7,1,3


Let's get the names of those datasets that have less than 4 valid $R^2$ values:

In [226]:
xgboost_single_output_invalid_datasets = xgboost_single_output_counts[
    xgboost_single_output_counts["R2_valid"] < 4
].index.to_list()
xgboost_single_output_invalid_datasets

['ist_shifts_3_dummy_adj',
 'basic_dummy',
 'basic_shifts_3_dummy',
 'complete_dummy',
 'complete_shifts_3_dummy',
 'ist_dummy',
 'ist_shifts_3_dummy',
 'no_ist_dummy',
 'no_ist_shifts_3_dummy',
 'no_ist_shifts_3_dummy_adj']

In [247]:
xgboost_single_output_best_datasets = xgboost_single_output_counts[
    xgboost_single_output_counts["R2_valid"] > 4
].index.to_list()
xgboost_single_output_best_datasets

['basic_adj',
 'basic_dummy_adj',
 'basic_shifts_3_adj',
 'complete_adj',
 'complete_dummy_adj',
 'complete_shifts_3_adj',
 'ist_adj',
 'ist_dummy_adj',
 'ist_shifts_3_adj',
 'no_ist_adj',
 'no_ist_dummy_adj',
 'no_ist_shifts_3_adj']

In [227]:
xgboost_single_output_counts = xgboost_single_output_df.merge(
    xgboost_single_output_counts, left_index=True, right_index=True
)

Analyze by dataset's completness:

In [228]:
xgboost_single_output_counts[
    ["type", "total_count", "total_podiums", "R2_valid"]
].groupby("type").agg(
    {"total_count": "sum", "total_podiums": "sum", "R2_valid": "mean"}
)

,total_count,total_podiums,R2_valid
type,,,
0,9,18,4.250
1,15,20,4.000
2,10,23,4.000
3,15,22,4.125


Analyze by presence of dummy variables:

In [229]:
xgboost_single_output_counts[
    ["dummy", "total_count", "total_podiums", "R2_valid"]
].groupby("dummy").agg(
    {"total_count": "sum", "total_podiums": "sum", "R2_valid": "mean"}
)

,total_count,total_podiums,R2_valid
dummy,,,
False,29,54,5.5000
True,20,29,2.6875


Analyze by number of shifted variables:

In [230]:
xgboost_single_output_counts[
    ["shifts", "total_count", "total_podiums", "R2_valid"]
].groupby("shifts").agg(
    {"total_count": "sum", "total_podiums": "sum", "R2_valid": "mean"}
)

,total_count,total_podiums,R2_valid
shifts,,,
False,29,56,4.5625
True,20,27,3.6250


Analyze by presence of adjacenct variables:

In [231]:
xgboost_single_output_counts[
    ["adj", "total_count", "total_podiums", "R2_valid"]
].groupby("adj").agg({"total_count": "sum", "total_podiums": "sum", "R2_valid": "mean"})

,total_count,total_podiums,R2_valid
adj,,,
False,16,16,2.0000
True,33,67,6.1875


## XGBoost Multi-Output Regression

See the results:

In [232]:
xgboost_multi_output_highlighted_df = highlight_max_min(xgboost_multi_output_df)
xgboost_multi_output_highlighted_df

,CDC_RMSE,CDC_R^2,CS_RMSE,CS_R^2,CUP_RMSE,CUP_R^2,ECPGUANYEM_RMSE,ECPGUANYEM_R^2,ERC_RMSE,ERC_R^2,OTH_RMSE,OTH_R^2,PP_RMSE,PP_R^2,PSC_RMSE,PSC_R^2,VOX_RMSE,VOX_R^2,adj,dummy,shifts,type
basic,6.912863,0.000000,3.965014,-3.819140,3.893145,-0.389481,2.706174,0.500688,5.114751,0.411082,1.824544,-2.168074,2.176135,0.396065,2.766259,0.806059,1.987060,0.211268,False,False,False,0
basic_adj,1.804412,0.000000,6.293718,-11.142670,2.757898,0.302803,2.322399,0.632267,2.709542,0.834718,2.049461,-2.997424,2.453194,0.232638,2.654518,0.821374,2.393367,-0.144083,True,False,False,0
basic_dummy,6.978979,0.000000,4.010917,-0.197497,4.815888,-6.994590,6.087027,-0.914426,15.456491,-2.103959,5.129501,-16.885397,3.615487,-2.535016,12.571362,-2.353762,4.292633,-8.122092,False,True,False,0
basic_dummy_adj,3.443830,0.000000,2.903251,0.372606,2.842560,-1.784718,5.542597,-0.587454,7.519377,0.265418,3.986608,-9.805424,1.779420,0.143819,8.737487,-0.620405,1.252809,0.222987,True,True,False,0
basic_shifts_3,6.904218,0.000000,4.297150,-4.660323,3.841069,-0.352557,4.135160,-0.165857,6.937369,-0.083416,1.358566,-0.756502,2.617992,0.125912,5.271760,0.295640,3.125693,-0.951643,False,False,True,0
basic_shifts_3_adj,3.037084,0.000000,8.281557,-20.024403,2.746335,0.308637,2.775309,0.474852,5.441625,0.333361,2.629305,-5.579350,2.589708,0.144858,4.279635,0.535712,2.963255,-0.753787,True,False,True,0
basic_shifts_3_dummy,8.388105,0.000000,16.005813,-2.381722,2.362189,-0.024438,2.387287,0.255059,10.727091,-1.963703,3.030764,-0.262899,2.892235,-1.575577,5.179332,-0.059032,3.120069,0.000000,False,True,True,0
basic_shifts_3_dummy_adj,8.341634,0.000000,14.193432,-1.659048,2.446374,-0.098574,2.039424,0.456226,12.242730,-2.860285,3.047085,-0.276366,3.903053,-3.689727,4.049446,0.352571,3.187998,0.000000,True,True,True,0
complete,6.747306,0.000000,4.079625,-4.101766,3.901088,-0.395156,2.649296,0.521456,4.916937,0.455754,1.683067,-1.695813,2.037605,0.470509,2.755683,0.807539,2.008371,0.194259,False,False,False,3
complete_adj,1.643768,0.000000,6.339746,-11.320926,2.798203,0.282276,2.145148,0.686257,2.544213,0.854273,1.891844,-2.406212,2.406441,0.261608,2.747918,0.808583,2.479356,-0.227769,True,False,False,3


In [233]:
xgboost_multi_output_counts = count_max_min_highlights_and_podiums(xgboost_multi_output_df)
xgboost_multi_output_counts

,max_R2_count,min_RMSE_count,R2_podium_count,RMSE_podium_count,R2_valid,total_count,total_podiums
basic_dummy_adj,3,2,3,2,3,5,5
complete_adj,3,2,3,2,5,5,5
basic_adj,2,1,3,2,5,3,5
no_ist_dummy_adj,1,1,2,3,3,2,5
basic_shifts_3_dummy,1,1,2,1,1,2,3
no_ist_shifts_3_dummy_adj,2,0,2,1,2,2,3
complete,2,0,2,0,4,2,2
complete_shifts_3,1,1,1,1,2,2,2
ist_shifts_3_dummy_adj,1,1,1,1,2,2,2
no_ist_shifts_3_adj,2,0,2,0,4,2,2


Let's get the names of those datasets that have less than 4 valid $R^2$ values:

In [234]:
xgboost_multi_output_invalid_datasets = xgboost_multi_output_counts[
    xgboost_multi_output_counts["R2_valid"] < 4
].index.to_list()
xgboost_multi_output_invalid_datasets

['basic_dummy_adj',
 'no_ist_dummy_adj',
 'basic_shifts_3_dummy',
 'no_ist_shifts_3_dummy_adj',
 'complete_shifts_3',
 'ist_shifts_3_dummy_adj',
 'ist_dummy_adj',
 'basic_shifts_3_dummy_adj',
 'complete_dummy_adj',
 'complete_shifts_3_dummy',
 'complete_shifts_3_dummy_adj',
 'ist_shifts_3',
 'ist_shifts_3_dummy',
 'no_ist_shifts_3',
 'basic_dummy',
 'basic_shifts_3',
 'complete_dummy',
 'ist_dummy',
 'no_ist_dummy',
 'no_ist_shifts_3_dummy']

In [248]:
xgboost_multi_output_best_datasets = xgboost_multi_output_counts[
    xgboost_multi_output_counts["R2_valid"] > 4
].index.to_list()
xgboost_multi_output_best_datasets

['basic', 'basic_adj', 'complete_adj', 'ist_adj', 'no_ist_adj']

In [235]:
xgboost_multi_output_counts = xgboost_multi_output_df.merge(
    xgboost_multi_output_counts, left_index=True, right_index=True
)

Analyze by dataset's completness:

In [236]:
xgboost_multi_output_counts[["type", "total_count", "total_podiums", "R2_valid"]].groupby("type").agg({     "total_count": "sum",     "total_podiums": "sum",     "R2_valid": "mean" })

,total_count,total_podiums,R2_valid
type,,,
0,15,20,2.625
1,9,20,2.625
2,11,25,2.500
3,14,18,2.500


Analyze by presence of dummy variables:

In [237]:
xgboost_multi_output_counts[
    ["dummy", "total_count", "total_podiums", "R2_valid"]
].groupby("dummy").agg(
    {"total_count": "sum", "total_podiums": "sum", "R2_valid": "mean"}
)

,total_count,total_podiums,R2_valid
dummy,,,
False,25,45,3.750
True,24,38,1.375


Analyze by number of shifted variables:

In [238]:
xgboost_multi_output_counts[
    ["shifts", "total_count", "total_podiums", "R2_valid"]
].groupby("shifts").agg(
    {"total_count": "sum", "total_podiums": "sum", "R2_valid": "mean"}
)

,total_count,total_podiums,R2_valid
shifts,,,
False,28,52,3.000
True,21,31,2.125


Analyze by presence of adjacenct variables:

In [239]:
xgboost_multi_output_counts[
    ["adj", "total_count", "total_podiums", "R2_valid"]
].groupby("adj").agg({"total_count": "sum", "total_podiums": "sum", "R2_valid": "mean"})

,total_count,total_podiums,R2_valid
adj,,,
False,19,30,1.6875
True,30,53,3.4375


In [258]:
xgboost_multi_output_filtered_df = xgboost_multi_output_df[
    ~xgboost_multi_output_df["dummy"] & ~xgboost_multi_output_df["shifts"]
]
xgboost_multi_output_resumed_df = highlight_max_min(xgboost_multi_output_filtered_df)
xgboost_multi_output_resumed_df

,CDC_RMSE,CDC_R^2,CS_RMSE,CS_R^2,CUP_RMSE,CUP_R^2,ECPGUANYEM_RMSE,ECPGUANYEM_R^2,ERC_RMSE,ERC_R^2,OTH_RMSE,OTH_R^2,PP_RMSE,PP_R^2,PSC_RMSE,PSC_R^2,VOX_RMSE,VOX_R^2,adj,dummy,shifts,type
basic,6.912863,0.000000,3.965014,-3.819140,3.893145,-0.389481,2.706174,0.500688,5.114751,0.411082,1.824544,-2.168074,2.176135,0.396065,2.766259,0.806059,1.987060,0.211268,False,False,False,0
basic_adj,1.804412,0.000000,6.293718,-11.142670,2.757898,0.302803,2.322399,0.632267,2.709542,0.834718,2.049461,-2.997424,2.453194,0.232638,2.654518,0.821374,2.393367,-0.144083,True,False,False,0
complete,6.747306,0.000000,4.079625,-4.101766,3.901088,-0.395156,2.649296,0.521456,4.916937,0.455754,1.683067,-1.695813,2.037605,0.470509,2.755683,0.807539,2.008371,0.194259,False,False,False,3
complete_adj,1.643768,0.000000,6.339746,-11.320926,2.798203,0.282276,2.145148,0.686257,2.544213,0.854273,1.891844,-2.406212,2.406441,0.261608,2.747918,0.808583,2.479356,-0.227769,True,False,False,3
ist,6.819112,0.000000,4.061365,-4.056201,3.883551,-0.382640,2.664062,0.516107,5.029999,0.430437,1.703780,-1.762573,2.064604,0.456384,2.740233,0.809692,2.005370,0.196665,False,False,False,1
ist_adj,1.709108,0.000000,6.393748,-11.531718,2.822666,0.269672,2.220390,0.663862,2.784576,0.825437,1.975504,-2.714128,2.478794,0.216539,2.702664,0.814835,2.464913,-0.213506,True,False,False,1
no_ist,6.837161,0.000000,4.052825,-4.034959,3.836103,-0.349062,2.605601,0.537111,5.359188,0.353447,1.779435,-2.013360,2.094839,0.440346,2.691774,0.816363,2.027200,0.179080,False,False,False,2
no_ist_adj,1.695294,0.000000,6.390995,-11.520931,2.770819,0.296255,2.218100,0.664555,2.661800,0.840491,1.967444,-2.683884,2.414236,0.256817,2.674611,0.818659,2.436174,-0.185374,True,False,False,2


## Identify Invalid Datasets

We define a dataset as invalid if it has less than 4 valid $R^2$ values in any of the models.

In [242]:
invalid_datasets = (
    set(linear_regression_invalid_datasets)
    & set(knn_invalid_datasets)
    & set(decision_tree_invalid_datasets)
    & set(xgboost_single_output_invalid_datasets)
    & set(xgboost_multi_output_invalid_datasets)
)
invalid_datasets

{'basic_dummy',
 'basic_shifts_3_dummy',
 'complete_dummy',
 'complete_shifts_3_dummy',
 'ist_dummy',
 'ist_shifts_3_dummy',
 'no_ist_dummy',
 'no_ist_shifts_3_dummy'}

## Identify Best Datasets

We define a dataset as the best if it has more than 4 valid $R^2$ values in all models.

In [250]:
best_datasets = (
    set(linear_regression_best_datasets)
    | set(knn_best_datasets)
    | set(decision_tree_best_datasets)
    | set(xgboost_single_output_best_datasets)
    | set(xgboost_multi_output_best_datasets)
)
best_datasets

{'basic',
 'basic_adj',
 'basic_dummy_adj',
 'basic_shifts_3_adj',
 'complete_adj',
 'complete_dummy_adj',
 'complete_shifts_3_adj',
 'ist_adj',
 'ist_dummy_adj',
 'ist_shifts_3_adj',
 'no_ist_adj',
 'no_ist_dummy_adj',
 'no_ist_shifts_3_adj'}

In [257]:
best_of_best_datasets = (
    set(linear_regression_best_datasets)
    # & set(knn_best_datasets)
    # & set(decision_tree_best_datasets)
    & set(xgboost_single_output_best_datasets)
    & set(xgboost_multi_output_best_datasets)
)
best_of_best_datasets

{'basic_adj', 'complete_adj', 'ist_adj', 'no_ist_adj'}

## Summary

In [283]:
def summary_dataframe(df):
    # Drop the unwanted columns
    df = df.drop(columns=['adj', 'dummy', 'shifts', 'type'])
    
    # Rename the columns
    # Replace CDC for Junts
    # Replace OTH for Altres
    # Replace CS for C's
    # Replace ECPGUANYEM for Comuns
    replacements = {
        'CDC': 'Junts',
        'OTH': 'Altres',
        'CS': "C's",
        'ECPGUANYEM': 'Comuns',
    }
    df.columns = df.columns.str.replace('CDC|OTH|CS|ECPGUANYEM', lambda m: replacements[m.group(0)], regex=True)
    
    # Rename the index
    df.index = ['Basic', 'Basic Adj.', 'Complete', 'Complete Adj.', 'IST', 'IST Adj.', 'No IST', 'No IST Adj.']
    
    # Sort the index based on the specified order
    order = ['Basic', 'No IST', 'IST', 'Complete', 'Basic Adj.', 'No IST Adj.', 'IST Adj.', 'Complete Adj.']
    df = df.reindex(order)
    
    # Convert numbers to string with comma as decimal separator and two decimal places
    df = df.applymap(lambda x: f'{x:.2f}'.replace('.', ','))
    
    # Rename the index label
    df.index.name = 'Dataset'
    
    return df

In [284]:
linear_regression_filtered_df = linear_regression_df[
    ~linear_regression_df["dummy"] & ~linear_regression_df["shifts"]
]
linear_regression_summary_df = summary_dataframe(linear_regression_filtered_df)
# linear_regression_summary_df = highlight_max_min(linear_regression_summary_df)
linear_regression_summary_df

C:\Users\guill\AppData\Local\Temp\ipykernel_8344\3639216771.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: f'{x:.2f}'.replace('.', ','))


,Junts_RMSE,Junts_R^2,C's_RMSE,C's_R^2,CUP_RMSE,CUP_R^2,Comuns_RMSE,Comuns_R^2,ERC_RMSE,ERC_R^2,Altres_RMSE,Altres_R^2,PP_RMSE,PP_R^2,PSC_RMSE,PSC_R^2,VOX_RMSE,VOX_R^2
Dataset,,,,,,,,,,,,,,,,,,
Basic,"13,11","0,00","13,81","-57,49","4,96","-1,25","6,31","-1,71","8,55","-0,65","1,42","-0,92","4,60","-1,70","5,49","0,24","1,21","0,71"
No IST,"13,64","0,00","13,72","-56,71","4,92","-1,22","6,28","-1,69","8,75","-0,72","1,41","-0,89","4,68","-1,79","5,46","0,24","1,21","0,71"
IST,"13,56","0,00","13,78","-57,20","4,89","-1,19","6,23","-1,65","8,65","-0,68","1,43","-0,94","4,61","-1,71","5,46","0,24","1,21","0,71"
Complete,"13,66","0,00","13,73","-56,76","4,90","-1,21","6,27","-1,68","8,75","-0,72","1,41","-0,90","4,67","-1,78","5,46","0,24","1,21","0,71"
Basic Adj.,"2,04","0,00","1,30","0,48","1,80","0,70","1,46","0,85","1,86","0,92","0,44","0,81","1,03","0,86","1,36","0,95","0,97","0,81"
No IST Adj.,"2,11","0,00","1,31","0,48","1,81","0,70","1,48","0,85","1,87","0,92","0,44","0,82","1,03","0,86","1,36","0,95","0,97","0,81"
IST Adj.,"2,11","0,00","1,29","0,49","1,80","0,70","1,46","0,86","1,86","0,92","0,44","0,82","1,02","0,87","1,35","0,95","0,97","0,81"
Complete Adj.,"2,11","0,00","1,30","0,48","1,81","0,70","1,46","0,85","1,86","0,92","0,44","0,82","1,03","0,86","1,36","0,95","0,97","0,81"


In [285]:
knn_filtered_df = knn_df[
    ~knn_df["dummy"] & ~knn_df["shifts"]
]
knn_summary_df = summary_dataframe(knn_filtered_df)
# knn_summary_df = highlight_max_min(knn_summary_df)
knn_summary_df

C:\Users\guill\AppData\Local\Temp\ipykernel_8344\3639216771.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: f'{x:.2f}'.replace('.', ','))


,Junts_RMSE,Junts_R^2,C's_RMSE,C's_R^2,CUP_RMSE,CUP_R^2,Comuns_RMSE,Comuns_R^2,ERC_RMSE,ERC_R^2,Altres_RMSE,Altres_R^2,PP_RMSE,PP_R^2,PSC_RMSE,PSC_R^2,VOX_RMSE,VOX_R^2
Dataset,,,,,,,,,,,,,,,,,,
Basic,"17,27","0,00","5,27","-7,51","5,65","-1,92","3,95","-0,06","2,73","0,83","1,71","-1,78","2,92","-0,09","4,27","0,54","3,39","-1,30"
No IST,"13,56","0,00","6,90","-13,57","4,03","-0,49","2,60","0,54","6,45","0,06","0,96","0,12","2,02","0,48","3,86","0,62","4,12","-2,39"
IST,"18,78","0,00","5,54","-8,40","5,63","-1,91","3,21","0,30","2,91","0,81","1,71","-1,77","2,84","-0,03","3,90","0,61","3,41","-1,32"
Complete,"13,46","0,00","6,83","-13,29","4,01","-0,47","2,57","0,55","6,47","0,06","0,95","0,15","2,00","0,49","3,82","0,63","4,13","-2,40"
Basic Adj.,"8,07","0,00","4,88","-6,29","5,30","-1,57","3,68","0,08","3,01","0,80","1,44","-0,96","2,65","0,10","4,79","0,42","2,98","-0,78"
No IST Adj.,"8,17","0,00","6,17","-10,67","4,46","-0,82","2,46","0,59","4,83","0,47","0,72","0,50","2,00","0,49","3,81","0,63","3,92","-2,06"
IST Adj.,"9,08","0,00","5,23","-7,39","4,73","-1,05","3,39","0,22","3,13","0,78","1,26","-0,51","2,77","0,02","4,98","0,37","2,89","-0,66"
Complete Adj.,"8,23","0,00","6,18","-10,69","4,46","-0,82","2,43","0,60","4,82","0,48","0,72","0,51","1,99","0,49","3,78","0,64","3,92","-2,07"


In [286]:
decision_tree_filtered_df = decision_tree_df[
    ~decision_tree_df["dummy"] & ~decision_tree_df["shifts"]
]
decision_tree_summary_df = summary_dataframe(decision_tree_filtered_df)
# decision_tree_summary_df = highlight_max_min(decision_tree_summary_df)
decision_tree_summary_df

C:\Users\guill\AppData\Local\Temp\ipykernel_8344\3639216771.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: f'{x:.2f}'.replace('.', ','))


,Junts_RMSE,Junts_R^2,C's_RMSE,C's_R^2,CUP_RMSE,CUP_R^2,Comuns_RMSE,Comuns_R^2,ERC_RMSE,ERC_R^2,Altres_RMSE,Altres_R^2,PP_RMSE,PP_R^2,PSC_RMSE,PSC_R^2,VOX_RMSE,VOX_R^2
Dataset,,,,,,,,,,,,,,,,,,
Basic,"5,98","0,00","5,20","-7,29","4,68","-1,01","6,18","-1,61","5,83","0,23","1,11","-0,17","2,45","0,24","7,59","-0,46","3,51","-1,46"
No IST,"4,39","0,00","6,37","-11,43","4,90","-1,20","5,76","-1,27","6,57","0,03","1,29","-0,57","2,49","0,21","9,06","-1,08","2,95","-0,74"
IST,"8,98","0,00","6,87","-13,49","5,31","-1,59","5,68","-1,20","7,58","-0,30","1,46","-1,03","3,31","-0,40","9,14","-1,12","2,78","-0,54"
Complete,"15,09","0,00","5,90","-9,65","5,10","-1,38","4,65","-0,47","4,85","0,47","1,44","-0,97","2,98","-0,14","6,84","-0,18","2,45","-0,20"
Basic Adj.,"8,36","0,00","5,30","-7,60","5,09","-1,37","4,95","-0,67","6,35","0,09","1,39","-0,83","3,34","-0,42","7,88","-0,57","2,76","-0,52"
No IST Adj.,"18,92","0,00","2,32","-0,65","5,59","-1,86","5,93","-1,39","6,79","-0,04","1,67","-1,66","3,38","-0,46","6,08","0,06","3,79","-1,88"
IST Adj.,"3,31","0,00","12,80","-49,26","3,88","-0,38","5,08","-0,76","5,19","0,39","1,93","-2,56","3,21","-0,31","6,10","0,06","4,58","-3,19"
Complete Adj.,"7,45","0,00","3,15","-2,04","4,05","-0,50","3,95","-0,07","6,75","-0,03","2,39","-4,41","3,46","-0,53","4,91","0,39","3,93","-2,09"


In [287]:
xgboost_single_output_filtered_df = xgboost_single_output_df[
    ~xgboost_single_output_df["dummy"] & ~xgboost_single_output_df["shifts"]
]
xgboost_single_output_summary_df = summary_dataframe(xgboost_single_output_filtered_df)
# xgboost_single_output_summary_df = highlight_max_min(xgboost_single_output_summary_df)
xgboost_single_output_summary_df

C:\Users\guill\AppData\Local\Temp\ipykernel_8344\3639216771.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: f'{x:.2f}'.replace('.', ','))


,Junts_RMSE,Junts_R^2,C's_RMSE,C's_R^2,CUP_RMSE,CUP_R^2,Comuns_RMSE,Comuns_R^2,ERC_RMSE,ERC_R^2,Altres_RMSE,Altres_R^2,PP_RMSE,PP_R^2,PSC_RMSE,PSC_R^2,VOX_RMSE,VOX_R^2
Dataset,,,,,,,,,,,,,,,,,,
Basic,"5,78","0,00","5,61","-8,63","4,55","-0,90","3,87","-0,02","3,27","0,76","0,93","0,17","1,82","0,58","3,66","0,66","1,51","0,55"
No IST,"5,82","0,00","5,34","-7,73","4,43","-0,80","3,79","0,02","3,64","0,70","1,02","0,02","1,81","0,58","3,55","0,68","1,53","0,53"
IST,"5,74","0,00","5,26","-7,48","4,37","-0,75","3,73","0,05","3,66","0,70","1,08","-0,11","1,83","0,57","3,67","0,66","1,56","0,51"
Complete,"5,77","0,00","5,07","-6,89","4,30","-0,70","3,74","0,05","3,93","0,65","1,16","-0,28","1,86","0,56","3,86","0,62","1,59","0,50"
Basic Adj.,"0,88","0,00","1,96","-0,18","1,84","0,69","1,73","0,80","2,55","0,85","0,62","0,64","1,41","0,75","2,27","0,87","1,43","0,59"
No IST Adj.,"0,87","0,00","1,93","-0,15","1,82","0,69","1,67","0,81","2,52","0,86","0,63","0,62","1,39","0,75","2,25","0,87","1,38","0,62"
IST Adj.,"0,99","0,00","1,99","-0,21","1,84","0,69","1,71","0,80","2,41","0,87","0,73","0,49","1,41","0,75","2,30","0,87","1,43","0,59"
Complete Adj.,"0,86","0,00","1,95","-0,17","1,80","0,70","1,67","0,81","2,49","0,86","0,62","0,64","1,40","0,75","2,25","0,87","1,38","0,62"


In [288]:
xgboost_multi_output_filtered_df = xgboost_multi_output_df[
    ~xgboost_multi_output_df["dummy"] & ~xgboost_multi_output_df["shifts"]
]
xgboost_multi_output_summary_df = summary_dataframe(xgboost_multi_output_filtered_df)
# xgboost_multi_output_summary_df = highlight_max_min(xgboost_multi_output_summary_df)
xgboost_multi_output_summary_df

C:\Users\guill\AppData\Local\Temp\ipykernel_8344\3639216771.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: f'{x:.2f}'.replace('.', ','))


,Junts_RMSE,Junts_R^2,C's_RMSE,C's_R^2,CUP_RMSE,CUP_R^2,Comuns_RMSE,Comuns_R^2,ERC_RMSE,ERC_R^2,Altres_RMSE,Altres_R^2,PP_RMSE,PP_R^2,PSC_RMSE,PSC_R^2,VOX_RMSE,VOX_R^2
Dataset,,,,,,,,,,,,,,,,,,
Basic,"6,91","0,00","3,97","-3,82","3,89","-0,39","2,71","0,50","5,11","0,41","1,82","-2,17","2,18","0,40","2,77","0,81","1,99","0,21"
No IST,"6,84","0,00","4,05","-4,03","3,84","-0,35","2,61","0,54","5,36","0,35","1,78","-2,01","2,09","0,44","2,69","0,82","2,03","0,18"
IST,"6,82","0,00","4,06","-4,06","3,88","-0,38","2,66","0,52","5,03","0,43","1,70","-1,76","2,06","0,46","2,74","0,81","2,01","0,20"
Complete,"6,75","0,00","4,08","-4,10","3,90","-0,40","2,65","0,52","4,92","0,46","1,68","-1,70","2,04","0,47","2,76","0,81","2,01","0,19"
Basic Adj.,"1,80","0,00","6,29","-11,14","2,76","0,30","2,32","0,63","2,71","0,83","2,05","-3,00","2,45","0,23","2,65","0,82","2,39","-0,14"
No IST Adj.,"1,70","0,00","6,39","-11,52","2,77","0,30","2,22","0,66","2,66","0,84","1,97","-2,68","2,41","0,26","2,67","0,82","2,44","-0,19"
IST Adj.,"1,71","0,00","6,39","-11,53","2,82","0,27","2,22","0,66","2,78","0,83","1,98","-2,71","2,48","0,22","2,70","0,81","2,46","-0,21"
Complete Adj.,"1,64","0,00","6,34","-11,32","2,80","0,28","2,15","0,69","2,54","0,85","1,89","-2,41","2,41","0,26","2,75","0,81","2,48","-0,23"
